<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# Autopilot--Pedestrian detects parking

## Loading the model

We assume that you have trained ``best_steering_model_xy.pth`` to the example folder in accordance with the instructions in the autopilot training model example.

> ### Execute the code below that we should be familiar with to initialize the PyTorch model.

In [ ]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

In [ ]:
from servoserial import ServoSerial
import threading
# Kill pthread
import inspect
import ctypes
import ipywidgets.widgets as widgets
from IPython.display import display
import time

controller = widgets.Controller(index=0)
display(controller)

### From the example of the object follower, the migration object detection function is demonstrated in this instance function.

Load object detection model

Add related algorithm methods

In [ ]:
from jetbot import ObjectDetector
global object_model
object_model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

def detection_center(detection):
    """Calculate the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)

def norm(vec):
    """Calculate the length of a two-dimensional vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Find the detection closest to the center of the image"""
    #Clear Cache
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection


Next, load the trained model from the ``best_steering_model_xy.pth`` file you uploaded.

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

Because the model weights are on the CPU memory, then the following code is still passed to the GPU device as always.

In [ ]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Creating a preprocessor function

We have loaded our model now, but there is a small problem. The format of our training model does not exactly match the format of the camera.
To do this, we need to do some pre-processing. This includes the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using the same parameters as we did during training (our camera provides values in the range [0, 255], and the training loaded image is in the range [0, 1], so we need to scale 255.0
3. Transfer data from CPU memory to GPU memory
4. Add a batch dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Now we have defined a preprocessing function that converts images from camera format to neural network input format.

Now, let's start showing our camera. After going through the last few examples, you should be familiar with this now.

In [ ]:
from IPython.display import display
# import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg
from servoserial import ServoSerial

# camera = Camera()
camera = Camera.instance(width=300, height=300)
servo_device = ServoSerial() 

# image_widget = ipywidgets.Image()
# traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
# display(image_widget)

image_widget = widgets.Image(format='jpeg', width=300, height=300)
display(image_widget)


def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 2048
    servo_device.Servo_serial_control(1, 2048)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 1300)

We will use the following code to print out the detected object.

In [ ]:
detections = object_model(camera.value)
print(detections)

Create a robot instance to drive the motor

In [ ]:
from jetbot import Robot

robot = Robot()

### Define the slider to control the JetBot
> ### Tip: We have configured initial values for the sliders. These initial values apply to our official Yahboom map, but if you want to train
These values may not apply to your dataset when you are on your own different road map, so increase or decrease the slider depending on your settings and environment.

1. Speed control (speed_gain_slider): To start JetBot, add ``speed_gain_slider``
2. Steering gain control (steering_gain_sloder): If you see that the JetBot is spinning, you need to reduce ``steering_gain_slider`` until it becomes smooth
3. Steering bias control (steering_bias_slider): If you see the JetBot biased to the far right or extreme left of the track, you should control this slider until the JetBot starts tracking the line or track at the center. This explains the motion deviation and camera offset

> ## Note: When you slide the related slider mentioned above, in order to obtain a smooth JetBot road following behavior, the slider value should not be moved very quickly. The slider value should be moved gently to adjust the motion parameters.

In [ ]:
import ipywidgets
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.51,description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.25, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.24, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

(Base speed))speed_gain_slider     -> 0.51(Outer corner)   0.60(Inner curve)    0.42    0.56

(P)steering_gain_slider          -> 0.25           0.37            0.22    0.27

(D)steering_dgain_slider         -> 0.24           0.24            0.10    0.13

(Base steering value)steering_bias_slider -> 0.00           0.00            0.00    0.00

The x and y sliders will display the predicted x, y values.

The steering slider will display our estimated steering value. This value is not the actual angle of the target, but a proportional value. When the actual angle is ``0``, this is 0, which increases/decreases as the actual angle increases/decreases.

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

Create a function that is called when the value of the camera changes. This function will perform the following steps

1. Preprocess camera image
2. Perform a neural network
3. Calculate the approximate steering value
4. Control the motor using proportional/differential control (PD)

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    
    # Calculate all detected objects
    detections = object_model(image)
    
    # Draw all detected objects
#     for det in detections[0]:
#         bbox = det['bbox']
#         cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
    
    #Select the object you want to track, select the value of label_widget, 1 is person
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == 1]
    
    #Mark the object to be tracked with green lines.
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
#         bbox = det['bbox']
#         cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), (int(300 * bbox[2]), int(300 * bbox[3])), (0, 255, 0), 4)
        ''' Stop the current Jetbot movement if it detects an object on the road that needs to be avoided '''
        robot.stop()
    else:
        xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
        x = xy[0]
        y = (0.5 - xy[1]) / 2.0

        x_slider.value = x
        y_slider.value = y

        speed_slider.value = speed_gain_slider.value

        angle = np.arctan2(x, y)
        pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
        angle_last = angle

        steering_slider.value = pid + steering_bias_slider.value

        #PID + Base speed + Gain
        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
execute({'new': camera.value})

We have created our neural network execution function, but now we need to attach it to the camera for processing.

> ## Tip: This code will move the robot!! Please put the Jetbot robot on the map you have trained before. If you collect the data and the model is well trained, you can see that Jetbot runs smoothly on the road. !

In [ ]:
camservoInitFunction()
camera.observe(execute, names='value')

If your Jetbot functions properly, it should now generate new commands for each new camera frame.

Now, you can put the JetBot on a track that has collected data and see if it can track the track.

If you want to stop this behavior, you can unload the binding of this callback function by executing the code for the cell below.

In [ ]:
camera.unobserve(execute, names='value')
time.sleep(0.1)
robot.stop()

In [ ]:
robot.stop()

In [ ]:
def jetbot_motion():
    count1 = count2 = count3 = count4 =  count5 = 0
    while 1:
        #Robot car left and right DC motor
        if controller.axes[1].value <= 0.1:
            if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1 
                and controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1):
                robot.stop()
            else:
                robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
            
            time.sleep(0.01)
        else:
            robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
            time.sleep(0.01)
          #Handle control code---2(Xbox360手柄)
#         if controller.axes[1].value <= 0:
#             robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
#             time.sleep(0.01)
#         else:
#             robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
#             time.sleep(0.01)
            
# thread1 = threading.Thread(target=jetbot_motion)
# thread1.setDaemon(False)
# thread1.start()

def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id") 
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [ ]:
thread1 = threading.Thread(target=jetbot_motion)
thread1.setDaemon(False)
thread1.start()

In [ ]:
stop_thread(thread1)

### Summary
If your JetBot doesn't track the road well, try to find out where it failed. The beauty is that we can collect more data for these failure scenarios, and JetBot should do better.